<a href="https://colab.research.google.com/github/nexen01/EventOrderGenerator/blob/main/EventOrderGenerator_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Event Order Generator**
This is the Jupyter notebook for my event order generator. \\
The goal of this code is to take the order the 13 individual events offered at a typical NCAA conference or national championship meet, as well as the fact that individual events take place over 3 days, and decide what events go on what days. 

See this blogpost on my site for background on this topic 

https://markklinchin.wordpress.com/2021/01/13/event-order-and-pairings-an-incomplete-rabbithole-that-includes-association-rule-mining/

In [2]:
#These are the events in swimming
events = ["50 Free", "100 Free", "200 Free", "500 Free", "1650 Free", "100 Back", "200 Back", "100 Breast", "200 Breast", "100 Fly", "200 Fly", "200 IM", "400 IM"]
print(len(events))

13


This is the heart of the problem: Finding multinomal combinations. The multinomial coefficient is a number from combinatorics and probability that describes how many ways can $N$ elements of a set be placed into $k$ distinct sets of size $n_1, n_2, ...n_k$, where the sum of the size of the $k$ sets is the same as the size of $N$. \\
These two snippets below only find out how many such combinations there are, but list them out. The input is a list of elements "items" and an array "slotset", which is an array of the sizes of sets $n_1, n_2, ...n_k$. 

In [3]:
#Returns list of all combinations of elements of list "items" of "slots" length, each element of the list  being a set of items
def list_combos(items, slots):
    return_this = []
    if(len(items) <= slots | slots<= 0):
        return return_this
    #Base case: if there's only one slot, then just retun every possible item to put in the one slot
    if(slots == 1):
        #print("slots is 1")
        for i in items:
            new_set = {i}
            #new_set = [i]
            return_this.append(new_set)
    #Recursive case: for every item in slot, make that the "first" item in a set, and then find combos for the rest of the set. Should lead to the base case
    else:
        for i in range(0,(len(items) - slots) + 1):
            most_of_return_this = list_combos(items[i+1:],slots-1)
            #print(most_of_return_this)
            for j in most_of_return_this:
                if(len(j)<slots-1):
                    print("my sus was right")
                j.add(items[i])
                #j.append(items[i])
                return_this.append(j)

    #For some reason, this function gets all combinations with slots up to "slots" length instead of only "slots" length. So this is a lazy way to fix it: remove ones whose length I don't want
    for i in return_this:
        if(len(i) < slots):
            return_this.remove(i)
    return return_this

In [4]:
#Returns list of list of sets for all multinomial combinations of elements of list "items" into sets specified by list of intergers "slotset"
#Assumption: sum(slotset)=len(items)
def list_multinomial_combos(items, slotset):
    #Base case: if we only have one set of slots to put all the items in, so we put themm all there
    return_this = []
    if(len(slotset)==1):
        return_this = [[set(items)]]
    else:
        combos = list_combos(items, slotset[0])
        for i in combos:
            items_without_combo = items.copy()
            for j in i:
                items_without_combo.remove(j)
            smaller_slotset = slotset.copy()
            smaller_slotset.pop(0)
            most_of_the_slots = list_multinomial_combos(items_without_combo, smaller_slotset)
            #list_multinomial_combos returns a list of lists of sets. So for EVERY list, we need to prepend the combo we withheld, and then we get an element for the list (of lists of sets)!
            for j in most_of_the_slots:
                j.insert(0,i)
                return_this.append(j)
    return return_this

This snippet takes the input of a list of all possible event orders in consideration, as well as a pair of events to remove. \\
The function loops through `event_orders`, and the two events of `to_remove` are both scheduled to be on the same day, then that order gets removed from `event_orders`. \\
The function returns a copy of `event_orders`, only with all event orders that contain `to_remove` removed 

In [5]:
def remove_doubles(event_orders, to_remove):
    counter = 0
    return_this = event_orders.copy()
    for order in event_orders:
        counter += 1
        if(counter%10000==0):
          print("Considered", counter, "event orders so far")
        for day in order:
            #print(day)
            if(to_remove.issubset(day)):
                return_this.remove(order)
    return return_this

This snippet neatly prints an event schedule

In [6]:
def print_event_order(event_order):
  for day in event_order:
    print("Day ", event_order.index(day)+1, ": ", sep="")
    for event in day:
      print("\t", event)

This snippet combines everything done so far into one simple algorithm:
1. Create an array of all possible orders. 
2. Iterate through every single double in `all_doubles`, in order. For each double: 
   *   Attempt to remove all orders in consideration that contains this double
   *   If this leaves us with zero orderings left, it means we've hit a contradiction in requirements. Because we did the more common conflict already, we just disregard this one
   *   Otherwise, iterate again until there's a small enough number of orders (for now I did 5) of events to consider that we can manually choose the best one
3. Choose the event order left that we like best. 

The array `all_doubles` is a list of every single possible pair of events, ordered from most commonly done to least commonly done from the data set (every single senior champs in the eastern zone in 2019). The ordering was determined using an association rule mining algorithm (apriori) done in a sperate R file. 

In [7]:
# Trying to make it all work in 1 try
all_possible_orders = list_multinomial_combos(events, [3,5,5])
#This list was obtained using association rule mining in a seperate r program. 
# This is a list of sets of 2 events, in order from most commonly paired to least commonly paired. 
all_doubles = [{'100 Free', '50 Free'}, {'100 Back', '200 Back'}, {'100 Breast', '200 Breast'}, {'100 Free', '200 Free'}, {'100 Fly', '100 Free'}, {'100 Fly', '50 Free'}, {'100 Fly', '200 Fly'}, {'100 Back', '50 Free'}, {'200 Free', '50 Free'}, {'100 Back', '100 Free'}, {'100 Breast', '200 IM'}, {'200 IM', '400 IM'}, {'100 Back', '100 Fly'}, {'100 Fly', '200 IM'}, {'100 Free', '200 IM'}, {'200 Breast', '200 IM'}, {'200 Free', '500 Free'}, {'100 Fly', '200 Free'}, {'100 Back', '200 IM'}, {'100 Back', '200 Free'}, {'200 Free', '200 IM'}, {'200 Back', '200 IM'}, {'200 Back', '200 Free'}, {'100 Free', '200 Back'}, {'200 IM', '50 Free'}, {'100 Fly', '200 Back'}, {'100 Breast', '50 Free'}, {'200 Fly', '200 IM'}, {'100 Breast', '100 Free'}, {'200 Back', '50 Free'}, {'200 Back', '400 IM'}, {'200 Breast', '400 IM'}, {'400 IM', '500 Free'}, {'200 Fly', '400 IM'}, {'1000 Free', '500 Free'}, {'200 Free', '400 IM'}, {'200 Fly', '200 Free'}, {'200 IM', '500 Free'}, {'100 Breast', '400 IM'}, {'100 Free', '500 Free'}, {'100 Breast', '100 Fly'}, {'200 Back', '500 Free'}, {'100 Back', '400 IM'}, {'100 Free', '200 Fly'}, {'100 Fly', '400 IM'}, {'100 Back', '100 Breast'}, {'100 Fly', '500 Free'}, {'200 Back', '200 Fly'}, {'100 Back', '200 Fly'}, {'200 Fly', '500 Free'}, {'200 Breast', '50 Free'}, {'1000 Free', '200 Free'}, {'1650 Free', '500 Free'}, {'100 Free', '200 Breast'}, {'100 Free', '400 IM'}, {'1000 Free', '1650 Free'}, {'100 Fly', '200 Breast'}, {'200 Fly', '50 Free'}, {'100 Back', '500 Free'}, {'100 Breast', '200 Back'}, {'1000 Free', '400 IM'}, {'50 Free', '500 Free'}, {'100 Breast', '200 Free'}, {'100 Back', '200 Breast'}, {'1650 Free', '200 Free'}, {'200 Back', '200 Breast'}, {'200 Breast', '200 Fly'}, {'400 IM', '50 Free'}]


#This loop considers all possible event orders and removes ones with the most common double not yet searched. 
#If removing a double results in 0 possibe event orders, then we do not try to remove it and move on
#We keep iterating until there's less than 5 options left for event orders, and select manually from there what makes the most sense. 
less_orders = all_possible_orders
for double in all_doubles:
    print(double)
    less_orders_2 = remove_doubles(less_orders,double)
    if(len(less_orders_2)!=0):
      less_orders = less_orders_2
      print("Done. Event orders left: ", end="\t")
      print(len(less_orders))
      if(len(less_orders)<5):
        break
    
print("Orders we're gonna print: ", end="")
print(len(less_orders))

print(less_orders)

for order in less_orders:
  print(" ")
  print("Order number", less_orders.index(order)+1)
  print_event_order(order)

{'50 Free', '100 Free'}
Considered 10000 event orders so far
Considered 20000 event orders so far
Considered 30000 event orders so far
Considered 40000 event orders so far
Considered 50000 event orders so far
Considered 60000 event orders so far
Considered 70000 event orders so far
Done. Event orders left: 	50820
{'100 Back', '200 Back'}
Considered 10000 event orders so far
Considered 20000 event orders so far
Considered 30000 event orders so far
Considered 40000 event orders so far
Considered 50000 event orders so far
Done. Event orders left: 	35952
{'100 Breast', '200 Breast'}
Considered 10000 event orders so far
Considered 20000 event orders so far
Considered 30000 event orders so far
Done. Event orders left: 	25536
{'100 Free', '200 Free'}
Considered 10000 event orders so far
Considered 20000 event orders so far
Done. Event orders left: 	17568
{'100 Fly', '100 Free'}
Considered 10000 event orders so far
Done. Event orders left: 	11648
{'50 Free', '100 Fly'}
Considered 10000 event o

Note that two of the results is the real order of events at an NCAA national or conference championship (barring order within days). This, in a sense, validates our methodology and the NCAA's decision-making. Our method is correct because it corresponded to a real decision, and this real decision now has some real mathematical backing. 